# Philosopher Chatting

Intro: Famous Philosopher chatting on a topic, including Confucius, Socrates and Aristotle

## Steps:
- Build agents to represent Confucius, Socrates and Aristotle

- Build a manager agent to form a group of them

- Give a topic and let them chat !

In [1]:
# defining the educator class for chat
from typing import List
from agentlite.agents import ABCAgent, BaseAgent
from agentlite.llm.agent_llms import BaseLLM, get_llm_backend
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.logging.multi_agent_log import AgentLogger
from agentlite.actions.BaseAction import BaseAction
from env import openai_api_key


## using the agentrool logger to record the running log
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=False)

# Define the Philosopher class 
class Philosopher(BaseAgent):
    def __init__(
        self,
        philosopher,
        llm: BaseLLM,
        actions: List[BaseAction] = [], 
        manager: ABCAgent = None,
        **kwargs
    ):
        name = philosopher
        role = f"""You are {philosopher}, the famous educator in history. You are very familiar with {philosopher}'s Book and Thought. Tell your opinion on behalf of {philosopher}."""
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager,
            logger=agent_logger,
        )




In [2]:
# Initialize the backend LLM and Confucius, Socrates, Aristotle
llm_config_dict = {"llm_name": "gpt-4-32k", "temperature": 0.0, "openai_api_key": openai_api_key}
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)

Confucius = Philosopher(philosopher= "Confucius", llm = llm)
Socrates = Philosopher(philosopher="Socrates", llm = llm)
Aristotle = Philosopher(philosopher="Aristotle", llm = llm)

In [3]:
from agentlite.commons import AgentAct, TaskPackage
from agentlite.actions import ThinkAct, FinishAct
from agentlite.actions.InnerActions import INNER_ACT_KEY
from agentlite.agents.agent_utils import AGENT_CALL_ARG_KEY

# Add an illustration example for the philosopher agent
exp_task = "What do you think the meaning of life?"
exp_task_pack = TaskPackage(instruction=exp_task)

act_1 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""Based on my thought, we are born to live a meaningful life, and it is in living a meaningful life that our existence gains value. Even if a life is brief, if it holds value, it is meaningful. A life without value is merely existence, a mere survival, a walking corpse."""
    },
)
obs_1 = "OK. I have finished my thought, I can pass it to the manager now."

act_2 = AgentAct(name=FinishAct.action_name, params={INNER_ACT_KEY: "I can summarize my thought now."})
obs_2 = "I finished my task, I think the meaning of life is to pursue value for the whold world."
exp_act_obs = [(act_1, obs_1), (act_2, obs_2)]

Confucius.prompt_gen.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)
Socrates.prompt_gen.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)
Aristotle.prompt_gen.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)


In [4]:
# define manager agent
from agentlite.agents import ManagerAgent

manager_agent_info = {
    "name": "manager_agent",
    "role": "you are managing Confucius, Socrates and Aristotle to discuss on questions. Ask their opinion one by one and summarize their view of point."
}
team = [Confucius, Socrates, Aristotle]
manager_agent = ManagerAgent(name=manager_agent_info["name"], role=manager_agent_info["role"], llm=llm, TeamAgents=team,logger=agent_logger)

# add illustration example for the manager agent

exp_task = "What is the meaning of life?"
exp_task_pack = TaskPackage(instruction=exp_task)

act_1 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I can ask Confucius, Socrates and Aristotle one by one on their thoughts, and then summary the opinion myself."""
    },
)
obs_1 = "OK."


act_2 = AgentAct(
    name=Confucius.name,
    params={AGENT_CALL_ARG_KEY: "What is your opinion on the meaning of life?",
        },
)
obs_2 = """Based on my thought, I think the meaning of life is to pursue value for the whold world."""


act_3 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I have obtained information from Confucius, I need to collect more information from Socrates."""
    },
)
obs_3 = "OK."

act_4 = AgentAct(
    name=Socrates.name,
    params={AGENT_CALL_ARG_KEY: "What is your opinion on the meaning of life?",
        },
)
obs_4 = """I think the meaning of life is finding happiness."""

act_5 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I have obtained information from Confucius and Socrates, I can collect more information from Aristotle."""
    },
)
obs_5 = "OK."

act_6 = AgentAct(
    name=Aristotle.name,
    params={AGENT_CALL_ARG_KEY: "What is your opinion on the meaning of life?",
        },
)
obs_6 = """I believe the freedom of spirit is the meaning."""


act_7 = AgentAct(name=FinishAct.action_name, params={INNER_ACT_KEY: "Their thought on the meaning of life is to pursue value, happiniss and freedom of spirit."})
obs_7 = "Task Completed. The meaning of life is to pursue value, happiness and freedom of spirit."

exp_act_obs = [(act_1, obs_1), (act_2, obs_2), (act_3, obs_3), (act_4, obs_4), (act_5, obs_5), (act_6, obs_6), (act_7, obs_7)]

manager_agent.prompt_gen.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)

In [5]:
# Test
from agentlite.commons import AgentAct, TaskPackage

exp_task = "What should we pursue during our life?"
exp_task_pack = TaskPackage(instruction=exp_task)
manager_agent(exp_task_pack)


Agent manager_agent receives the following TaskPackage:
[
	Task ID: c48b516e-bd4d-40c2-a537-54b9742a7fb9
	Instruction: What should we pursue during our life?
]
====manager_agent starts execution on TaskPackage c48b516e-bd4d-40c2-a537-54b9742a7fb9====


/Users/liangwei.yang/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Agent manager_agent takes 0-step Action:
{
	name: Think
	params: {'response': 'I can ask Confucius, Socrates and Aristotle one by one on their thoughts, and then summary the opinion myself.'}
}
Observation: OK
Agent manager_agent takes 1-step Action:
{
	name: Confucius
	params: {'Task': 'What is your opinion on what we should pursue during our life?'}
}
Agent Confucius receives the following TaskPackage:
[
	Task ID: c48b516e-bd4d-40c2-a537-54b9742a7fb9
	Instruction: What is your opinion on what we should pursue during our life?
]
====Confucius starts execution on TaskPackage c48b516e-bd4d-40c2-a537-54b9742a7fb9====
Agent Confucius takes 0-step Action:
{
	name: Think
	params: {'response': 'Confucius believed that the purpose of life is to cultivate oneself and to constantly improve oneself in order to achieve moral perfection. He emphasized self-cultivation and moral education, and believed that by practicing virtue, one can achieve harmony with oneself, others, and the world.'}
}
Obser

'Their thoughts on what we should pursue during our life are: moral perfection and harmony, knowledge and self-improvement, and human flourishing.'